In [1]:
import pandas as pd
import numpy as np

In [2]:
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=column_names)
df.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


We have the movie id but not the names, we'll now merge the list above with the movir id titles csv

In [3]:
movie_titles_df = pd.read_csv('Movie_Id_Titles')
movie_titles_df.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [4]:
merged_df = pd.merge(df, movie_titles_df, on='item_id')
merged_df.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


In [9]:
n_users = merged_df.user_id.nunique()
n_items = merged_df.item_id.nunique()
print('Number of users is ' + str(n_users))
print('Number of movies is ' + str(n_movies))

Number of users is 944
Number of movies is 1682


We will now will now split the data but only into two sets (not xtrain,ytrain, ytest,ytest)

In [8]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(merged_df, test_size=0.25)

We wil be using memory based collaboratie filteringm

An example of item-item collaborative filtering
![item-item collaborative filtering](https://www.liip.ch/content/4-blog/20160616-recommender-system-slack-pandas-flask/BLOG_CCA_10-2.png)

An example of user-item collaborative filtering
![user-item collaborative filtering](https://liip.rokka.io/www_inarticle/0e575a55ff1076359ddde8fdcd5f806fb7da6d94/blog-cca-11-2.jpg)

Now we will create a user-item matrix to train the model.

In [11]:
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]
    

In [17]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

Now that we have created matrics we can now make predictions.

In [27]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [28]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

Now we can evaluate. We will evaluate using RMSE

In [29]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [30]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))


ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.